In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
## provide the path for testing cofing file and tained model form colab
net = cv2.dnn.readNetFromDarknet("C:\\Users\\Vishnu\\Downloads\\yolov3_custom_testing.cfg",r"C:\\Users\\Vishnu\\Downloads\\yolov3_custom_4000.weights")

In [ ]:
### Change here for custom classes for trained model 

classes = ['Drowning','Swimming']

In [ ]:
#test by image -  also saves output
import cv2
import numpy as np

# Load the YOLOv3 model
## provide the path for testing config file and trained model from colab
net = cv2.dnn.readNetFromDarknet("C:\\Users\\Vishnu\\Downloads\\yolov3_custom_testing.cfg", r"C:\\Users\\Vishnu\\Downloads\\yolov3_custom_4000.weights")
# net = cv2.dnn.readNet("path/to/weights", "path/to/config")

# Load the class labels
classes = ['Drowning', 'Swimming']

# Set the confidence threshold and NMS threshold
confidence_threshold = 0.5
nms_threshold = 0.3

# Load the input image
image = cv2.imread("C:\\Users\\Vishnu\\Downloads\\underwater.jpg")

# Preprocess the image
blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)

# Set the input blob for the network
net.setInput(blob)

# Forward pass through the network
output_layers_names = net.getUnconnectedOutLayersNames()
layer_outputs = net.forward(output_layers_names)

# Process the outputs
boxes = []
confidences = []
class_ids = []

for output in layer_outputs:
    for detection in output:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]

        if confidence > confidence_threshold:
            # Scale the bounding box coordinates
            box = detection[0:4] * np.array([image.shape[1], image.shape[0], image.shape[1], image.shape[0]])
            (center_x, center_y, width, height) = box.astype("int")

            # Calculate the top-left corner coordinates
            x = int(center_x - (width / 2))
            y = int(center_y - (height / 2))

            boxes.append([x, y, int(width), int(height)])
            confidences.append(float(confidence))
            class_ids.append(class_id)

# Apply non-maximum suppression
indices = cv2.dnn.NMSBoxes(boxes, confidences, confidence_threshold, nms_threshold)

# Draw bounding boxes and labels
for i in indices.flatten():
    (x, y, w, h) = boxes[i]
    label = classes[class_ids[i]]
    confidence = confidences[i]
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.putText(image, f"{label}: {confidence:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Save the output image
output_image_path = "C:\\Users\\Vishnu\\Downloads\\output_image.jpg"
cv2.imwrite(output_image_path, image)

# Display the output image
cv2.imshow("Output", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
#By live_webcam- working perfect
cap = cv2.VideoCapture(0)

while 1:
    _, img = cap.read()
    img = cv2.resize(img,(1280,720))
    hight,width,_ = img.shape
    blob = cv2.dnn.blobFromImage(img, 1/255,(416,416),(0,0,0),swapRB = True,crop= False)

    net.setInput(blob)

    output_layers_name = net.getUnconnectedOutLayersNames()

    layerOutputs = net.forward(output_layers_name)

    boxes =[]
    confidences = []
    class_ids = []

    for output in layerOutputs:
        for detection in output:
            score = detection[5:]
            class_id = np.argmax(score)
            confidence = score[class_id]
            if confidence > 0.7:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * hight)
                w = int(detection[2] * width)
                h = int(detection[3]* hight)
                x = int(center_x - w/2)
                y = int(center_y - h/2)
                boxes.append([x,y,w,h])
                confidences.append((float(confidence)))
                class_ids.append(class_id)


    indexes = cv2.dnn.NMSBoxes(boxes,confidences,.5,.4)

    boxes =[]
    confidences = []
    class_ids = []

    for output in layerOutputs:
        for detection in output:
            score = detection[5:]
            class_id = np.argmax(score)
            confidence = score[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * hight)
                w = int(detection[2] * width)
                h = int(detection[3]* hight)

                x = int(center_x - w/2)
                y = int(center_y - h/2)



                boxes.append([x,y,w,h])
                confidences.append((float(confidence)))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes,confidences,.8,.4)
    font = cv2.FONT_HERSHEY_PLAIN
    colors = np.random.uniform(0,255,size =(len(boxes),3))
    if  len(indexes)>0:
        for i in indexes.flatten():
            x,y,w,h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = str(round(confidences[i],2))
            color = colors[i]
            cv2.rectangle(img,(x,y),(x+w,y+h),color,2)
            cv2.putText(img,label + " " + confidence, (x,y+400),font,2,color,2)

    cv2.imshow('img',img)
    if cv2.waitKey(1) == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

In [ ]:
#by_video
import cv2
import numpy as np

# Load the YOLOv3 model
## provide the path for testing cofing file and tained model form colab
net = cv2.dnn.readNetFromDarknet("C:\\Users\\Vishnu\\Downloads\\yolov3_custom_testing.cfg",r"C:\\Users\\Vishnu\\Downloads\\yolov3_custom_4000.weights")
#net = cv2.dnn.readNet("path/to/weights", "path/to/config")

# Load the class labels
classes = ['Drowning','Swimming']

# Set the confidence threshold and NMS threshold
confidence_threshold = 0.5
nms_threshold = 0.3

# Load the input video
video_path = "C:\\Users\\Vishnu\\Downloads\\drowning.mp4"
cap = cv2.VideoCapture(video_path)

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define output video writer
#output_path = "D:\\1vishnu_vit\\ai\\output.mp4"
output_folder = "D:\\1vishnu_vit\\ai\\"
output_filename = "output.mp4"
output_path = output_folder + output_filename
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
#output = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
#output = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (frame_width, frame_height))
frame_counter = 0

while cap.isOpened():
    # Read the frame
    ret, frame = cap.read()

    if not ret:
        break

    # Preprocess the frame
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)

    # Set the input blob for the network
    net.setInput(blob)

    # Forward pass through the network
    output_layers_names = net.getUnconnectedOutLayersNames()
    layer_outputs = net.forward(output_layers_names)

    # Process the outputs
    boxes = []
    confidences = []
    class_ids = []

    for output in layer_outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > confidence_threshold:
                # Scale the bounding box coordinates
                box = detection[0:4] * np.array([frame_width, frame_height, frame_width, frame_height])
                (center_x, center_y, width, height) = box.astype("int")

                # Calculate the top-left corner coordinates
                x = int(center_x - (width / 2))
                y = int(center_y - (height / 2))

                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply non-maximum suppression
    indices = cv2.dnn.NMSBoxes(boxes, confidences, confidence_threshold, nms_threshold)
    if len(indices) > 0:
        for i in indices.flatten():
            (x, y, w, h) = boxes[i]
            label = classes[class_ids[i]]
            confidence = confidences[i]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, f"{label}: {confidence:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    
    # Write the processed frame to the output video
    #output.write(frame)
    output_filename = f"frame_{frame_counter}.jpg"
    cv2.imwrite(output_filename, frame)
    frame_counter += 1

    # Display the processed frame
    cv2.imshow("Output", frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
output.release()
cv2.destroyAllWindows()
